# fact_messaging_daily

### Importación de librerías

In [3]:
import yaml
import pandas as pd
from sqlalchemy import create_engine

### Conexión a base y bodega de datos

In [4]:
with open('config.yaml') as f:
    config = yaml.safe_load(f)
    configFuente = config['fuente']
    configBodega = config['bodega']

urlFuente = f"{configFuente['driver']}://{configFuente['user']}:{configFuente['password']}@{configFuente['host']}:{configFuente['port']}/{configFuente['db']}"
urlBodega = f"{configBodega['driver']}://{configBodega['user']}:{configBodega['password']}@{configBodega['host']}:{configBodega['port']}/{configBodega['db']}"

src = create_engine(urlFuente)
etl = create_engine(urlBodega)

### Extracción y transformación de datos

In [5]:

mensajeria_servicio = pd.read_sql_table('mensajeria_servicio', src)
clientes_usuarioaquitoy = pd.read_sql_table('clientes_usuarioaquitoy', src)

mensajeria_servicio.drop(columns=["id", "descripcion", "nombre_solicitante", "hora_solicitud", "fecha_deseada", "hora_deseada", "nombre_recibe", "telefono_recibe", "descripcion_pago", "ida_y_regreso", "activo", "novedades", "cliente_id", "destino_id", "mensajero_id", "origen_id", "tipo_pago_id", "tipo_servicio_id", "tipo_vehiculo_id", "prioridad", "ciudad_destino_id", "ciudad_origen_id", "hora_visto_por_mensajero", "visto_por_mensajero", "descripcion_multiples_origenes", "mensajero2_id", "mensajero3_id", "multiples_origenes", "asignar_mensajero", "es_prueba", "descripcion_cancelado"], inplace=True)

clientes_usuarioaquitoy.drop(columns=["ciudad_id", "user_id", "telefono", "area_id", "token_Firebase", "lider"], inplace=True)

fact_messaging_daily = pd.merge(mensajeria_servicio, clientes_usuarioaquitoy, left_on='usuario_id', right_on='id', how='inner')

fact_messaging_daily.drop(columns=["usuario_id", "id"], inplace=True)

total_day = fact_messaging_daily.groupby(['fecha_solicitud', 'cliente_id', 'sede_id']).size().reset_index(name='total_day')
fact_messaging_daily = fact_messaging_daily.merge(total_day, on=['fecha_solicitud', 'cliente_id', 'sede_id'])
fact_messaging_daily = fact_messaging_daily.drop_duplicates(subset=['fecha_solicitud', 'cliente_id', 'sede_id'])

fact_messaging_daily.rename(columns={'fecha_solicitud': 'fecha'}, inplace=True)

dim_fecha = pd.read_sql_table('dim_fecha', etl)

fact_messaging_daily = fact_messaging_daily.merge(dim_fecha[['fecha', 'key_dim_fecha']], on='fecha', how='left')
fact_messaging_daily.drop(columns=['fecha'], inplace=True)
fact_messaging_daily.rename(columns={'key_dim_fecha': 'key_fecha', 'cliente_id': 'key_cliente', 'sede_id': 'key_sede',}, inplace=True)

fact_messaging_daily["key_fact_messaging_daily"] = range(1, len(fact_messaging_daily) + 1)

fact_messaging_daily

,key_cliente,key_sede,total_day,key_fecha,key_fact_messaging_daily
0,5,6,1,299,1
1,5,5,1,299,2
2,5,5,1,301,3
3,2,2,2,311,4
4,5,8,1,311,5
...,...,...,...,...,...
4289,2,2,1,334,4290
4290,8,37,1,439,4291
4291,5,5,1,293,4292
4292,2,2,1,276,4293


### Carga de datos

In [6]:
fact_messaging_daily.to_sql("hecho_entrega_mensajeria_diario", etl, index=False, if_exists="replace")

294